In [ ]:
%%capture
!pip install pyspark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Audio Classification with PySpark').config("spark.some.config.option", "some-value").getOrCreate()

In [ ]:
dt = spark.read.csv("drive/MyDrive/UNI-AMRITA-SEM2/MLforBigData/MLBD_Project/MLBD_Dataset/train.csv",  header=True)
dt.show(5)

+---+----------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+----------+-----------+----------+----------+----------+----------+----------+-----------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+-----+
|_c0|         0|        1|         2|         3|         4|         5|         6|         7|         8|         9|        10|         11|        12|         13|        14|        15|        16|        17|        18|         19|        20|       21|       22|       23|       24|       25|       26|       27|       28|       29|       30|       31|       32|       33|       34|       35|       36|       37|       38|       39|label|
+---+----------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----

In [ ]:
dt.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- 0: string (nullable = true)
 |-- 1: string (nullable = true)
 |-- 2: string (nullable = true)
 |-- 3: string (nullable = true)
 |-- 4: string (nullable = true)
 |-- 5: string (nullable = true)
 |-- 6: string (nullable = true)
 |-- 7: string (nullable = true)
 |-- 8: string (nullable = true)
 |-- 9: string (nullable = true)
 |-- 10: string (nullable = true)
 |-- 11: string (nullable = true)
 |-- 12: string (nullable = true)
 |-- 13: string (nullable = true)
 |-- 14: string (nullable = true)
 |-- 15: string (nullable = true)
 |-- 16: string (nullable = true)
 |-- 17: string (nullable = true)
 |-- 18: string (nullable = true)
 |-- 19: string (nullable = true)
 |-- 20: string (nullable = true)
 |-- 21: string (nullable = true)
 |-- 22: string (nullable = true)
 |-- 23: string (nullable = true)
 |-- 24: string (nullable = true)
 |-- 25: string (nullable = true)
 |-- 26: string (nullable = true)
 |-- 27: string (nullable = true)
 |-- 28: string (n

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import FloatType, StringType, IntegerType
dt = dt.select(*(col(c).cast("float").alias(c) for c in dt.columns[:]))
dt = dt.withColumn("label",col("label").cast(IntegerType()))

In [ ]:
dt.printSchema()

root
 |-- _c0: float (nullable = true)
 |-- 0: float (nullable = true)
 |-- 1: float (nullable = true)
 |-- 2: float (nullable = true)
 |-- 3: float (nullable = true)
 |-- 4: float (nullable = true)
 |-- 5: float (nullable = true)
 |-- 6: float (nullable = true)
 |-- 7: float (nullable = true)
 |-- 8: float (nullable = true)
 |-- 9: float (nullable = true)
 |-- 10: float (nullable = true)
 |-- 11: float (nullable = true)
 |-- 12: float (nullable = true)
 |-- 13: float (nullable = true)
 |-- 14: float (nullable = true)
 |-- 15: float (nullable = true)
 |-- 16: float (nullable = true)
 |-- 17: float (nullable = true)
 |-- 18: float (nullable = true)
 |-- 19: float (nullable = true)
 |-- 20: float (nullable = true)
 |-- 21: float (nullable = true)
 |-- 22: float (nullable = true)
 |-- 23: float (nullable = true)
 |-- 24: float (nullable = true)
 |-- 25: float (nullable = true)
 |-- 26: float (nullable = true)
 |-- 27: float (nullable = true)
 |-- 28: float (nullable = true)
 |-- 29: float

In [ ]:
from collections import defaultdict
data_types = defaultdict(list)
for entry in dt.schema.fields:
  data_types[str(entry.dataType)].append(entry.name)
print(data_types)

defaultdict(<class 'list'>, {'FloatType()': ['_c0', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39'], 'IntegerType()': ['label']})


In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
stage_onehot_enc =  [OneHotEncoder(inputCol='label', outputCol='label_onehot_enc')]

In [ ]:
from pyspark.ml import Pipeline
ppl = Pipeline(stages=stage_onehot_enc)
dt_trans = ppl.fit(dt).transform(dt)
dt_trans.show(10)

+---+----------+----------+----------+----------+-----------+----------+----------+----------+-----------+----------+----------+-----------+----------+-----------+----------+----------+----------+-----------+-----------+-----------+----------+---------+---------+---------+----------+----------+---------+---------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+-----+----------------+
|_c0|         0|         1|         2|         3|          4|         5|         6|         7|          8|         9|        10|         11|        12|         13|        14|        15|        16|         17|         18|         19|        20|       21|       22|       23|        24|        25|       26|       27|        28|       29|       30|       31|       32|       33|       34|       35|       36|       37|       38|       39|label|label_onehot_enc|
+---+----------+----------+----------+----------+-----------+----------+--------

In [ ]:
train_set, test_set =dt.randomSplit([0.75,0.10])

In [ ]:
train_set.show(5)

+---+----------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+----------+-----------+----------+----------+----------+----------+----------+-----------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+-----+
|_c0|         0|        1|         2|         3|         4|         5|         6|         7|         8|         9|        10|         11|        12|         13|        14|        15|        16|        17|        18|         19|        20|       21|       22|       23|       24|       25|       26|       27|       28|       29|       30|       31|       32|       33|       34|       35|       36|       37|       38|       39|label|
+---+----------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
features = VectorAssembler(inputCols=dt.columns[:41], outputCol='features')
model_rf = RandomForestClassifier(featuresCol='features', labelCol='label')
pipe_lr = Pipeline(stages = [features, model_rf]) 

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label')
parameters = ParamGridBuilder().addGrid(model_rf.bootstrap, [True,False]).addGrid(model_rf.maxDepth, [5,10,20,30]).build()
cv = CrossValidator(estimator=pipe_lr,estimatorParamMaps=parameters,evaluator= evaluator)

In [ ]:
cvModel = cv.fit(train_set)

In [ ]:
cvModel.bestModel.stages[-1]

RandomForestClassificationModel: uid=RandomForestClassifier_76951da4c32f, numTrees=20, numClasses=4, numFeatures=41

In [ ]:
cvModel.avgMetrics

[0.40704131259686815,
 0.40704131259686815,
 0.40704131259686815,
 0.40704131259686815,
 0.29141013307679975,
 0.29141013307679975,
 0.29141013307679975,
 0.29141013307679975]

In [ ]:
predict = cvModel.transform(test_set)

f1 = evaluator.evaluate(predict, {evaluator.metricName:'f1'})

accuracy = evaluator.evaluate(predict, {evaluator.metricName:'accuracy'})

print(f'F1 score:{f1}')

print(f'Accuracy score:{accuracy}')

F1 score:0.5317460317460319
Accuracy score:0.4444444444444444


In [ ]:
#model.save("path")